## Thar Desert Dust
Markus Petters, June 17, 2018

This notebook derives the effective surface area for the Thar Desert dust sample. 

### Problem Statement
Desert dust was size classified (D = [200, 300, 400, 500, 600] nm) and the monodisperse size was sampled onto a filter. Simulateneously the number concentration of particles was monitored using a condensation particle counter. What is the surface area per particle of dust sampled?

### Approach
A suspension of 0.1 w/w of dust in ultrapure water was prepared at the same concentration as for the filter collection. The sample was atomized, dried, and charge neutralized using an X-ray neutralizer. The size distribution was measured using the high-flow DMA column operated in scanning mobility particle sizer mode. The size distribution D > 130 nm is fit to a Junge model. The Junge model is used together with the code _DifferentialMobilityAnalyzers.jl_ to to determine the surface area contribution by multiply charged particles to a selected size. 

In [2]:
using Plots, Plots.PlotMeasures, DifferentialMobilityAnalyzers, Interpolations, LambertW, LsqFit, DataFrames
plotlyjs()
include("load.jl")
include("invertsmps.jl");

### Load Raw Data
The experiment was performed on June 15, 2018. Three size spectra between 17:07-17:21 correspond to Thar dust. Before sampling the spectra the atomimzer was cleaned with ultrapure water. The atomizer was fed with a syringe pump operating at 0.4 mL min-1. The dust sample was added and the size distribution monitored until no further change in the distribution was observed. After 17:21 the system was set to monitor laboratory air to test of the inversion algorithm.

In [3]:
ss,ee = DateTime(2018,6,15,17,7,0), DateTime(2018,6,15,17,25,0)
tdma = readtdma("data/HFDMA/20180615/")
tᵗᵈ = (tdma->tdma.t).(tdma)
jj = find((tᵗᵈ .≥ ss) .& (tᵗᵈ .≤ ee));

Loading data...100%|████████████████████████████████████| Time: 0:00:02


### HFDMA Transmission Model
The HFDMA Transmission was measured and fit to the expression below. This expression captures deviation from ideal transmission as a function of size.

In [4]:
leff = 190.0
u = (Λ,D) -> @. D*leff/Λ.qsa
Peff = u -> @. 0.82*exp(-11.5u)+0.1*exp(-70.0u)+0.03*exp(-180.0u)+0.02*exp(-340.0u)
Tᴰᴹᴬ = (Λ,dp) -> 0.9*clean(Peff(u(Λ,dab(Λ,dp*1e-9))));

### Size Distribution Inversion
The size distribution is inverted using the package _DifferentialMobilityAnalyzers.jl_. The function 

```julia
𝕄,ℝ,mlabel,Λ,δ = makeresponse(tdma,ss,ee,signal,qsh,qsa,τᵈ=τᵈ,DMA=DMA,tᶜ=1);
```

takes the raw data, applies the calibrated delay time between effective voltage and detector (11 s), and applies the integration time for each size bin. The DMA was operated a 1 L min-1 sample flow and 9 L min-1 sheath flow.

```julia
T = (i,k,Λ,δ) -> δ.Ω(Λ,i,k).*δ.Tc(k,δ.Dp).*Tᴰᴹᴬ(Λ,δ.Dp)
```

defines the transmission model for the HFDMA from which the convolution matrix 𝐀 is derived as described in Petters (2018). The size distribution is inverted using a fixed regularization parameter λ = 0.08 that was found from the L-curve method. ℕ is an array of inverted size distributions.

In [5]:
qsa,qsh,signal,τᵈ,DMA = 1lpm,9lpm,:Ncount1,11,:HFDMA
𝕄,ℝ,mlabel,Λ,δ = makeresponse(tdma,ss,ee,signal,qsh,qsa,τᵈ=τᵈ,DMA=DMA,tᶜ=1);
T = (i,k,Λ,δ) -> δ.Ω(Λ,i,k).*δ.Tc(k,δ.Dp).*Tᴰᴹᴬ(Λ,δ.Dp)
𝐀 = (hcat(map(i->Σ(k->T(i,k,Λ,δ),Λ.m),1:length(δ.Dp))...))';
λ = 0.08
ℕ = map(𝕣->invertfast(𝕣.N,δ,𝐀,λ),ℝ);

### Model
The size distribution is fit for D > 130 nm. The lower cutoff was selected because the smallest dust sample for ice nucleation was 200 nm and because dissolved materials dominate the atomized sample at smaller diameters. The size distribution is fit to a Junge distribution. The Junge distribution is then generated on a grid of a high-flow DMA operated at 9:3 L min-1, as was the case for the filter collections. Then the size distribution and the model are graphed. 

In [6]:
qsa,qsh = 3lpm, 9lpm
t,p = 295.15, 1e5                         
leff,m = 0.0,6                            
r₁,r₂,l = 0.05,0.058,0.6                  
bins,z₁,z₂ = 256, dtoz(Λ,1000e-9),dtoz(Λ,120e-9) 
Λ₁ = DMAconfig(t,p,qsa,qsh,r₁,r₂,l,leff,:-,m)  
δ₁ = setupDMA(Λ, z₁, z₂, bins);    

In [7]:
ii = ℕ[3].Dp .> 120
model(x,p) = p[1]./x.^p[2] 
fit = curve_fit(model, ℕ[3].Dp[ii], ℕ[3].S[ii], [4e8,2.5])
p = fit.param;
function junge(p,Λ,δ)
    model(x,p) = p[1]./x.^p[2] 
    S = model(δ.Dp,p)
    N = S.*δ.ΔlnD
    return SizeDistribution(p,δ.De,δ.Dp,δ.ΔlnD, S,N,:Junge)    
end
𝕗 = junge(p,Λ₁,δ₁);

In [8]:
figure("Nimbus Sans L", 2, 3.5, 2.5, 8)
p2 = plot(ℕ[1].Dp,ℕ[1].S, xaxis = :log10, xlabel = "Diameter (nm)",xlim = [100,1000], 
     ylabel = "dN/dlnD (cm<sup>-3</sup>)", label = mlabel[1], ylim = [0,3000])
p2 = plot!(ℕ[2].Dp,ℕ[2].S,label = mlabel[2])
p2 = plot!(ℕ[3].Dp,ℕ[3].S,label = mlabel[3])
p2 = plot!(𝕗.Dp,𝕗.S, color = :black, lw = 2, label = "Fit", left_margin = 30px)

__Fig. 1. __ Size distribution of atomized Thar desert dust sample. The three consecutive distributions are in excellent agreement. The fit is selected to represent the distribution for the model calculations.

### Effective Surface Area
The effective surface area is computed as described in Notebook S4 in Petters (2018). 

In [9]:
T = (zˢ,k,Λ,δ) -> δ.Ω(Λ,δ.Z,zˢ/k).*δ.Tc(k,δ.Dp)
Nz = zˢ -> Σ(k->T(zˢ,k,Λ₁,δ₁).*𝕗, Λ₁.m)
Az = zˢ -> (π*(𝕗.Dp*1e-3).^2).*(Σ(k->T(zˢ,k,Λ₁,δ₁).*𝕗, Λ₁.m))
function areaperparticle(Dm)
    zˢ = dtoz(Λ₁, Dm*1e-9)  # Compute corresponding electrical mobility
    𝕟ᴺ = Nz(zˢ)            # Number distribution
    𝕟ᴬ = Az(zˢ)            # Area distribution  
    return sum(𝕟ᴬ.N)./sum(𝕟ᴺ.N)
end
Dm = [200,300,400,500,600,700]
effectiveA = map(Dm->areaperparticle(Dm),Dm)
simpleA = map(Dm->(π*(Dm*1e-3).^2),Dm)
DataFrame(Dm=Dm, simpleA=simpleA,effectiveA=effectiveA,ratio=round.(effectiveA./simpleA,3))

,Dm,simpleA,effectiveA,ratio
1,200,0.125664,0.174757,1.391
2,300,0.282743,0.405136,1.433
3,400,0.502655,0.666156,1.325
4,500,0.785398,0.943487,1.201
5,600,1.13097,1.19754,1.059
6,700,1.53938,1.53826,0.999


__Table 1. __ The table shows diameter Dm (μm), simple surface area per particle (μm² cm⁻³), effective surface area per particle (μm² cm⁻³) accounting for the contribution of multiply charged particles, and the enhancement ratio (-). Simple surface area per particle is computed as $A = \pi D^2$.

### Inversion Test
Spectrum #5 is the HFDMA and CPC measuring laboratory air. This is a quick test to check if the inverted solution is reasonable. Note that the integrated number concetrations approximately agree. 

In [10]:
s = 5
figure("Nimbus Sans L", 2, 5, 2, 8)
p1 = plot(ℝ[s].Dp,ℝ[s].N, xaxis = :log10, xlabel = "Diameter (nm)",ylabel = "Concentration (cm<sup>-3</sup>)", 
    label = "ℝ "*mlabel[s], color = :black)

p2 = plot(ℕ[s].Dp,ℕ[s].S, xaxis = :log10, xlabel = "Diameter (nm)", ylabel = "dN/dlnD (cm<sup>-3</sup>)", 
    label = "ℕ "*mlabel[s], color = :black)

plot(p2,p1, layout = grid(1,2), left_margin = 40px)

__Fig. 2.__ Size distribution of laboratory air. Left: HFDMA raw response function. Right: Inverted size distribution after regularization.

### Number Concentration Closure
Inverted number concentration is compared to that measured by a CPC. This is done to ensure that the inversion is reasonble.

In [11]:
data = readsmps("data/SMPS/20180615/");
field = f->(data->getfield(data,f)).(data)
smps = f->squeeze(hcat(field(f)'...), 1)
tsec = smps(:tsec)
Nserial = smps(:Nserial)
Ncount1 = smps(:Ncount1)
hh = floor.(Int, tsec./3600)
min = floor.(Int, (tsec .% 3600)/60)
ss = convert(Array{Int},tsec .% 60)
t = DateTime.(2018,6,15,hh,min,ss)

ss,ee = DateTime(2018,6,15,17,25,0), DateTime(2018,6,15,17,27,0)
ii = find((t .> ss) .& (t .< ee))

@sprintf("Ninverted = %i cm-3;  Ncpc = %i cm-3", sum(ℕ[s].N),mean(Nserial[ii]))

"Ninverted = 1879 cm-3;  Ncpc = 1728 cm-3"

The CPC concentration is taken from the second data system. The inverted concentration slightly varies with the regularization parameter, but not in a significant manner. A CPC intercomparison was performed recently showing that the HFDMA CPC measured ~9% higher than the other CPC. The result is well within the acceptable uncertainty of CPCs and size distribution measurement from SMPS.

### L-curve
The L-curve for the regularized inversion is shown for completeness. See Notebook S05 in Petters (2018). Note that the result is not sensitive to small changes in λ.

In [ ]:
figure("Helvetica", 2, 3.5, 2, 8)

function mlcurve(λ₁::Float64, λ₂::Float64; n::Int = 10)
    λs = logspace(log10(λ₁), log10(λ₂), n)
    L1, L2 = reginv(λs, r=:L1L2)
    return L1, L2, λs
end

λ₁,λ₂=0.005,1.0
R = ℝ[5].N
setupRegularization(𝐀,eye(length(R)),R,inv(δ.𝐒)*R)
L1,L2,λs = mlcurve(λ₁,λ₂;n=1000)
ii = 523
plot(L1,L2, color = :black)
plot!([L1[ii], L1[ii]], [L2[ii], L2[ii]], marker = :square, color = RGBA(0.8,0,0,0.5), ylabel = "𝐈*(𝕟-𝕟ᵢ)",
            label = "λ ="*(@sprintf("%.1e", λs[ii])),lw = 0, ms = 4, xlabel = "𝐀*(𝕟-𝕣)", left_margin = 30px)

__ Fig. 3. __ L-curve for spectrum 5 shown in Fig. 2. The value λ=0.08 was used on the inversion.

### References

Petters, M. D. (2018) A language to simplify computation of differential mobility analyzer response functions, Aerosol Science & Technology. 

Source code for _DifferentialMobilityAnalyzers.jl_:  https://github.com/mdpetters/DifferentialMobilityAnalyzers.jl